### Scripts for preprocessing all log-files
Author: Amer Altizini
Date: 06.12.2022
Information: Cells that are within the same section need to run subsequently

In [19]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import folium
import os
import glob

***
### Plotting Data
***

In [25]:
filepath_sensors = './Data/input/'
output_filepath = './Data/output/'

# speech_column = ['NTPTime','speech','manipulation','robot']
# gesture_column = ['NTPTime','gesture','manipulation','robot']

if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [26]:
# Merge accelerometer and annotation data

In [32]:
for participant_filepath in glob.glob(filepath_sensors + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Accelerometer.csv'.format(participant_filepath))
    accelerometer = accelerometer.rename(columns={'z': 'ACC_Z', 'y': 'ACC_Y', 'x': 'ACC_X'})
    gyroscope = pd.read_csv('{}Gyroscope.csv'.format(participant_filepath))
    gyroscope = gyroscope.rename(columns={'z': 'GYRO_Z', 'y': 'GYRO_Y', 'x': 'GYRO_X'})
    annotation = pd.read_csv('{}Annotation.csv'.format(participant_filepath))
    display(accelerometer)
    print("{}.csv".format(filepath_sensors))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    participant_filepath_output = output_filepath + participant + '/'
    if not os.path.exists(participant_filepath_output):
        os.makedirs(participant_filepath_output)
    merged = pd.merge_ordered(accelerometer, gyroscope, on=["time"])
    merged = pd.merge_ordered(merged, annotation, on=["time"])
    merged.text.ffill(inplace=True)
    merged.to_csv(participant_filepath_output + "ACC_GYRO_annot.csv", index=False)
    display(merged)

,time,seconds_elapsed,ACC_Z,ACC_Y,ACC_X
0,1670358941526095000,0.237095,-0.051105,-0.008579,0.004120
1,1670358941536018400,0.247018,-0.039528,0.005057,0.005821
2,1670358941545941800,0.256942,-0.038113,0.017600,0.022669
3,1670358941555864800,0.266865,-0.030545,0.011678,0.020639
4,1670358941565788200,0.276788,-0.034830,0.006293,0.019006
...,...,...,...,...,...
2925,1670358970551983000,29.262983,0.024566,-0.003135,-0.007701
2926,1670358970561906400,29.272906,0.002955,-0.010218,-0.000554
2927,1670358970571829800,29.282830,0.000550,-0.010013,-0.002948
2928,1670358970581753300,29.292753,-0.006525,-0.019336,-0.000592


./Data/input/.csv


'2022-12-06_20-35-41'

,time,seconds_elapsed_x,ACC_Z,ACC_Y,ACC_X,seconds_elapsed_y,GYRO_Z,GYRO_Y,GYRO_X,seconds_elapsed,text
0,1670358941526095000,0.237095,-0.051105,-0.008579,0.004120,0.237095,-0.001198,-0.001598,0.000666,NaN,NaN
1,1670358941536018400,0.247018,-0.039528,0.005057,0.005821,0.247018,-0.000133,-0.000533,0.001731,NaN,NaN
2,1670358941545941800,0.256942,-0.038113,0.017600,0.022669,0.256942,-0.000133,-0.000533,0.000666,NaN,NaN
3,1670358941555864800,0.266865,-0.030545,0.011678,0.020639,0.266865,-0.001198,-0.001598,0.000666,NaN,NaN
4,1670358941565788200,0.276788,-0.034830,0.006293,0.019006,0.276788,-0.001198,-0.000533,0.000666,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2928,1670358970551983000,29.262983,0.024566,-0.003135,-0.007701,29.262983,-0.001198,-0.000533,0.000666,NaN,Yyy
2929,1670358970561906400,29.272906,0.002955,-0.010218,-0.000554,29.272906,-0.001198,-0.000533,0.000666,NaN,Yyy
2930,1670358970571829800,29.282830,0.000550,-0.010013,-0.002948,29.282830,-0.001198,-0.000533,-0.000399,NaN,Yyy
2931,1670358970581753300,29.292753,-0.006525,-0.019336,-0.000592,29.292753,-0.001198,-0.000533,0.000666,NaN,Yyy


In [40]:
# path_E4 = "./data/output/"

for participant_filepath in glob.glob(filepath_sensors + '*/', recursive=True):
    accelerometer = pd.read_csv('{}Accelerometer.csv'.format(participant_filepath))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')
    accelerometer.index = pd.to_datetime(accelerometer['time'], unit = 'ns')

    fig = go.Figure()

    for axis in ['x', 'y', 'z']:
        fig.add_trace(go.Scatter(x = accelerometer.index, y = accelerometer[axis], name = axis))
    fig.update_layout(legend_title_text = "ACC")
    fig.update_xaxes(title_text="Timestamp_" + participant)
    fig.update_yaxes(title_text="ACC")
    fig.show()

'2022-12-06_20-35-41'

In [10]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Gyroscope.csv')

df.index = pd.to_datetime(df['time'], unit = 'ns')

fig = go.Figure()

for axis in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x = df.index, y = df[axis], name = axis))

fig.show()

In [11]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Magnetometer.csv')

df.index = pd.to_datetime(df['time'], unit = 'ns')

fig = go.Figure()

for axis in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x = df.index, y = df[axis], name = axis))

fig.show()

In [7]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Location.csv')
df.index = pd.to_datetime(df['time'], unit='ns')

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['speed'], mode='markers',
              error_y={'type': 'data', 'array': df['speedAccuracy']}))

fig.show()

In [23]:
df = pd.read_csv("./Data/2022-12-06_17-32-02/Location.csv")
coords = [(row.latitude, row.longitude) for _, row in df.iterrows()]

my_map = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=16)
folium.PolyLine(coords, color="blue", weight=5.0).add_to(my_map)
my_map

***
### Aligning & Interpolating Measurements Across Sensors
***

In [16]:
df_gyro = pd.read_csv('./Data/2022-12-06_17-32-02/Gyroscope.csv')
df_acce = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')

for df in [df_gyro, df_acce]:
    df.index = pd.to_datetime(df['time'], unit = 'ns')

df = df_gyro.join(df_acce, lsuffix = '_gyro', rsuffix = '_acce', how = 'outer').interpolate()
df

,time_gyro,seconds_elapsed_gyro,z_gyro,y_gyro,x_gyro,time_acce,seconds_elapsed_acce,z_acce,y_acce,x_acce
time,,,,,,,,,,
2022-12-06 18:19:28.736621800,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.301622,0.000000,0.000000,0.000000
2022-12-06 18:19:28.746593300,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.311593,-0.037347,-0.002901,0.005993
2022-12-06 18:19:28.756671500,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.321671,-0.082677,0.022784,0.017380
2022-12-06 18:19:28.766749700,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.331750,-0.150242,0.028249,-0.006868
2022-12-06 18:19:28.776924400,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.341924,-0.248892,-0.003977,-0.054344
...,...,...,...,...,...,...,...,...,...,...
2022-12-06 18:19:43.798824000,1.670351e+18,15.363824,0.013715,0.048468,-0.000399,1.670351e+18,15.343973,0.327040,-0.105531,0.047844
2022-12-06 18:19:43.808749600,1.670351e+18,15.373750,0.021305,0.063381,-0.016378,1.670351e+18,15.343973,0.327040,-0.105531,0.047844
2022-12-06 18:19:43.818675000,1.670351e+18,15.383675,0.028761,0.081490,-0.043009,1.670351e+18,15.343973,0.327040,-0.105531,0.047844


***
### Fourier Transforms
***

In [18]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Gravity.csv')
ts = pd.to_datetime(df['time'], unit = 'ns')
period = np.nanmedian(ts.diff().dt.total_seconds())

sp = np.fft.fft(df['x'])
freq = np.fft.fftfreq(len(df.index), period)

mask = (freq >= 0) &(freq < 1)

fig = go.Figure()
fig.add_trace(go.Scatter(x = freq[mask], y = [np.linalg.norm(s) for s in sp[mask]]))
fig.show()